In [ ]:
import torch
import torch.nn as nn

from tqdm import tqdm
import wandb


In [ ]:
net = Transformer()
optim = torch.optim.AdamW(params = net.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs=100
vocab_size = 20000
num_classes = 4
context_length = 2048

Xtest = torch.randint(0,vocab_size,(20,context_length))
Ytest = torch.randint(0,num_classes,(20,))

Xtrain = torch.randint(0,vocab_size, (20,context_length))
Ytrain = torch.randint(0,num_classes,(20,))

dataset = torch.utils.data.TensorDataset(Xtrain,Ytrain)
dataloader = torch.utils.data.DataLoader(dataset,batch_size=4,shuffle=True)

: 

In [ ]:
#wandb.init(project="cell_type_classification",config={})

for e in tqdm(range(epochs)):
    net.eval()
    predTest = net(Xtest)
    predTrain = net(Xtrain)
    lossTest = loss_fn(predTest,Ytest).item()
    loss_fn.zero_grad()
    lossTrain = loss_fn(predTrain,Ytrain).item()
    loss_fn.zero_grad()
    
    # wandb.log({"train_loss": lossTrain, "test_loss": lossTest})

    net.train()
    for i, (batch_X, batch_Y) in enumerate(dataloader):
        pred = net(batch_X)
        loss = loss_fn(pred,batch_Y)
        loss.backward()
        optim.step()
        optim.zero_grad()

  0%|          | 0/100 [00:00<?, ?it/s]

torch.Size([20, 2048, 512])
torch.Size([20, 2048, 512])
torch.Size([4, 2048, 512])
torch.Size([4, 2048, 512])
torch.Size([4, 2048, 512])
torch.Size([4, 2048, 512])
torch.Size([4, 2048, 512])


  1%|          | 1/100 [01:20<2:13:21, 80.83s/it]